In [ ]:
using Revise
using BenchmarkTools

In [ ]:
using Support
using GCTGMT

In [ ]:
name_to_set_to_element_ = Dict(
    splitpath(path)[end] => read_gmt(path) for
    path in read_directory("/Users/kwat/Desktop/gene_sets/")
)

In [ ]:
using FeatureSetEnrichment

In [ ]:
method_ = collect(keys(score_set_new(["a", "b"], [-1.0, 1.0], ["a"]; plot = false)))

In [ ]:
benchmarks_directory_path = "/Users/kwat/Desktop/benchmarks/"

In [ ]:
using DataIO

In [ ]:
function load_benchmark(benchmark_directory_path)

    element_, score_ =
        eachcol(read_data(joinpath(benchmark_directory_path, "gene_x_score.tsv")))

    json_dict = read_json(joinpath(benchmark_directory_path, "gene_sets.json"))

    set_to_element_ = Dict{String,Vector{String}}()

    for gmt in json_dict["gene_sets_tested"]

        merge!(set_to_element_, name_to_set_to_element_[gmt])

    end

    return element_, score_, set_to_element_, json_dict["gene_sets_positive"]

end

In [ ]:
compute = true

In [ ]:
using Normalization

In [ ]:
benchmark_ = Vector{String}()

n_gene_ = Vector{Int64}()

benchmark_x_method_row_ = Vector{Vector{Float64}}()

In [ ]:
for benchmark_directory_path in read_directory(benchmarks_directory_path)

    println(splitpath(benchmark_directory_path)[end])

    element_, score_, set_to_element_, benchmark_set_ =
        load_benchmark(benchmark_directory_path)

    if compute

        set_to_method_to_statistic = score_set_new(element_, score_, set_to_element_)

        # TODO: try without collect
        set_ = collect(keys(set_to_method_to_statistic))

        set_x_method = convert_vector_of_vector_to_matrix([
            collect(values(set_to_method_to_statistic[set_[set_i]])) for
            set_i = 1:length(set_)
        ])

    end

    for set in benchmark_set_

        benchmark = string(splitpath(benchmark_directory_path)[end], '.', set)

        push!(benchmark_, benchmark)

        n_gene = length(set_to_element_[set])

        push!(n_gene_, n_gene)

        if compute

            benchmark_x_method_row = Vector{Float64}()

            # TODO: simplify loop
            for (method_i, set_score_) in enumerate(eachcol(set_x_method))

                is_negative_ = set_score_ .< 0

                is_positive_ = 0 .<= set_score_

                set_negative_ = set_[is_negative_]

                set_positive_ = set_[is_positive_]

                negative_ =
                    .-normalize(.-set_score_[is_negative_], "1224") / sum(is_negative_)

                positive_ = normalize(set_score_[is_positive_], "1224") / sum(is_positive_)

                i_negative = findfirst(set_negative_ .== set)

                i_positive = findfirst(set_positive_ .== set)

                if i_negative != nothing

                    benchmark_score = negative_[i_negative]

                elseif i_positive != nothing

                    benchmark_score = positive_[i_positive]

                else

                    error(method_[method_i], " returned ", benchmark_score)

                end

                push!(benchmark_x_method_row, benchmark_score)

            end

            push!(benchmark_x_method_row_, benchmark_x_method_row)

        end

    end

end

n_benchmark = length(benchmark_)

In [ ]:
using CSV
using DataFrames

In [ ]:
benchmark_x_method_path = "benchmark_x_method.tsv"

axis0_name = "Benchmark"

if compute

    df = DataFrame(convert_vector_of_vector_to_matrix(benchmark_x_method_row_), method_)

    insertcols!(df, 1, axis0_name => benchmark_)

    CSV.write(benchmark_x_method_path, df)

end

benchmark_x_method = read_data(benchmark_x_method_path)

In [ ]:
#method_ = ["is ks < area", "is ks < extreme", "a idrsw <> area", "a idrdw <> area"]

benchmark_x_method = Matrix(benchmark_x_method[!, method_])

In [ ]:
function peek_benchmark(id::String)::Nothing

    benchmark, set = split(id, '.'; limit = 2)

    element_, score_, set_to_element_, benchmark_set_ =
        load_benchmark(joinpath(benchmarks_directory_path, benchmark))

    score_set_new(element_, score_, set_to_element_[set])

    return nothing

end

In [ ]:
benchmark_x_method_metric = Matrix{Float64}(undef, size(benchmark_x_method))

for benchmark_i = 1:length(benchmark_)

    base_score = benchmark_x_method[benchmark_i, 1]

    score_ = benchmark_x_method[benchmark_i, :]

    if base_score < 0

        peek_benchmark(benchmark_[benchmark_i])

        metric_ = score_ .< base_score

    else

        metric_ = base_score .< score_

    end

    benchmark_x_method_metric[benchmark_i, :] = metric_

end

In [ ]:
function sum_sort_print(m::Matrix)::Nothing

    for (n_better, method) in
        zip(sort_like(Float64.(sum(eachrow(m))), method_; r = true)...)

        println(Int64(n_better), "    ", method)

    end

end

In [ ]:
sum_sort_print(benchmark_x_method_metric)

In [ ]:
using Plot

In [ ]:
use_style!()

In [ ]:
using Plotly

In [ ]:
x = benchmark_x_method[:, 1]

for method_i = 2:length(method_)

    display(
        plot_x_y(
            [x, x],
            [x, benchmark_x_method[:, method_i]];
            text_ = [benchmark_, benchmark_],
            mode_ = ["line", "markers"],
            layout = Layout(
                xaxis_title_text = method_[1],
                yaxis_title_text = method_[method_i],
            ),
        ),
    )

end